# Long Call Option

### I. Strategy Overview

This is the most basic option strategy composed of a single long call leg.

#### Payoff
Option Payoff or Profit and Loss (P&L) diagrams help us understand where our options strategies win or lose money at expiration based on different stock price points.

<!-- ![](long_call.jpg) -->
<img src='../img/long_call.jpg'/>

#### Features (optional):
* Long Delta: option value increases as the underlying price goes up
* Short Theta: option value decreases as time decays
* Long volatility: option value increases as the implied volatility hikes

### II. Strategy Setup
* Observe the historical price of selected underlying
* Find avaialble expiration dates for options
* Select an expiration date, option type and stike price for the option strategy
* Select entrance and exit point (simplified by choosing future enter and exit dates)

In [1]:
import sys  
sys.path.insert(0, 'C:\\Users\\chena\\Desktop\\Trading\\StrategicOptions\\')
import datetime
from Functions.mainprg import GetDays, GetPayOff, GetBackTesting, GetPriceHistory
import pandas as pd
import numpy as np
from datetime import timedelta

ImportError: cannot import name 'GetDays' from 'Functions.mainprg' (C:\Users\chena\Desktop\Trading\StrategicOptions\Functions\mainprg.py)

####  a. Select underlying by ticker

In [ ]:
Ticker = 'MSFT'
GetPriceHistory(Ticker).plot_price_history()

#### b. Find expiration dates for testing period

In [ ]:
Start_Date = datetime.datetime(2018, 1, 4)
End_Date = datetime.datetime(2018,6, 30)

GetDays(Ticker, Start_Date, End_Date, []).expiration_dates

#### c. Select an expiration date, choose option type (call) and strike price

In [ ]:
expiration = datetime.datetime(2018, 3, 18)
option_type = ['C']  # C(all), P(ut)
action = ['L']  #L(ong), S(hort)

# Change the strike price to see how it affects the strategy
# strike_price = [85]  # At the money
strike_price = [75]  # Deep in the money
#strike_price = 95  # Far out the money

#### d. Choose option enter and exit dates

In [ ]:
entry_date = datetime.datetime(2018, 1, 4)
exit_date = datetime.datetime(2018, 3, 1)

### III. Visualization
#### a. Generate option(s) price and payoff

In [ ]:
payoff = GetPayOff(Ticker, strike_price, entry_date, expiration, option_type, action)
print('Option Premium = '+"${:.2f}".format(payoff.option_price[0]))
payoff.plot_payoff()

#### * Note that regardless of the strike/expiration date you choose, the probability of profit for longing an option is always less than 50%

##### b. Compare returns by holding the option vs. buy&hold the underlying

In [ ]:
bt1 = GetBackTesting(Ticker, strike_price, expiration, entry_date, exit_date, option_type, action)
bt1.plot_price_history()

In [ ]:
bt1.plot_option_pnl()

#### c. Strategy Summary

In [ ]:
bt1.strategy_summary

### III. Recurring Strategy Setup

#### a. Choose the backtesting period for the recurring startegy

In [ ]:
Start_Date = datetime.datetime(2016, 1, 4)
End_Date = datetime.datetime(2018,8, 30)

expiry = GetDays(Ticker, Start_Date, End_Date, []).expiration_dates
trading_days = GetDays(Ticker, Start_Date, End_Date, []).trading_days
trading_days['Year-Month'] = pd.to_datetime(trading_days['Date']).dt.to_period('M') + 1  # +1 to find the expiration date in the next month

dte_seeker = pd.merge(trading_days, expiry, on='Year-Month', how='left')
dte_seeker['DTE'] = (pd.to_datetime(dte_seeker['Expiration Date']) - pd.to_datetime(dte_seeker['Date'])).dt.days
dte_seeker.dropna(inplace=True)

#### b. Set trading rules

#### Simplified (current):
* Enter: 43 days before expriation
* Exit: 21 days before expiration
* Strike prices = enter day underlying price - \\$5

#### Realistic (example):
* IV < 20%
* Expiration date < 60 & Expiration date > 45
* VIX < 20
* other technical signals.

In [ ]:
entry_dates = pd.to_datetime(dte_seeker[dte_seeker['DTE']==43]['Date'].reset_index(drop=True))
expiration_dates = pd.to_datetime(dte_seeker[dte_seeker['DTE']==43]['Expiration Date'].reset_index(drop=True))
exit_dates = entry_dates + timedelta(days=24)
expiration_dates = pd.to_datetime(expiration_dates)

#### c. Run recursive analysis

In [ ]:
strategy_performance = pd.DataFrame([])
strat_sum_for_stats = []
strike_price = [80]
i = 0
for _ in entry_dates:
    bt = GetBackTesting(Ticker, strike_price, expiration_dates[i], entry_dates[i], exit_dates[i], option_type, action)
    strategy_performance = strategy_performance.append(bt.strategy_summary)
    strat_sum_for_stats.append(bt.strat_sum_for_stats)
    i += 1
    end_of_period_spot = bt.underlying_price_truncated_['close'][-1]
    strike_price = [end_of_period_spot-5]
    
win_rate = "{:.2%}".format(strategy_performance['Win'].sum()/len(strategy_performance['Win']))

In [ ]:
strike_price = np.array(strike_price)

In [ ]:
win_rate

In [ ]:
strategy_performance.reset_index(drop=True, inplace=True)
strategy_performance

In [ ]:
strategy_performance['Cost Basis']
avg_cost =strategy_performance['Cost Basis'].str.strip('$').astype(float).mean()
print('Avg Cost: ', "${:.2f}".format(avg_cost))

In [ ]:
strategy_performance['P&L']
pnl_sum =strategy_performance['P&L'].str.strip('$').astype(float).sum()
print('Total Profit: ', "${:.2f}".format(pnl_sum))